In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pylab import rcParams
import os
import warnings
import scipy.signal as signal
warnings.filterwarnings('ignore')
from datetime import datetime
rcParams['figure.figsize'] = 12,4
rcParams['savefig.dpi'] = 300
from detecta import detect_onset, detect_peaks

logo_path = 'logo/'
now = datetime.now().strftime('%Y%m%d')


def lowpass_filter(data, sr, cut_off, order):
    nyq = 0.5 * sr
    b, a = signal.butter(order, cut_off/nyq, btype = 'low')
    lp_df = signal.filtfilt(b, a, data)
    return lp_df

In [4]:
day = 20240213
file_list = sorted(glob(f'data/{day}/*csv'))
players = pd.read_excel(f'data/{day}/player information.xlsx', index_col='name')

,weight,height,fc_frame,br_frame,ball_vel,Unnamed: 6
name,,,,,,
n51_005,72,178,485,530,129,80.0
n51_008,72,178,603,635,128,79.4
n51_011,72,178,589,628,132,82.2
n52_004,73,170,520,553,115,71.4
n52_005,73,170,395,428,115,71.3
n52_006,73,170,386,418,115,71.4
n53_009,65,165,555,604,110,68.4
n53_011,65,165,702,743,111,68.8
n53_024,65,165,380,426,110,68.5


In [9]:
wrong = []
data = {}
for file in file_list:
    df = pd.read_csv(file)
    sessions = df.sessionid.unique()

    for session in sessions:
        d = df[df['sessionid'] == session].sort_values('frame').reset_index(drop=True)
        side = d['handedness'][0]
        trial = str(session.split('_')[-1]).zfill(3)
        name = d['athlete_name'][0].replace(' ','')
        
        weight, height, fc_frame, br_frame, ball_vel, _ = players.loc[f"{name}_{trial}"]
        fc_frame = int(fc_frame) ; br_frame = int(br_frame)
        weight, height = round(weight), round(height)
        newcols = []
        for col in d.columns:
            split_col = col.split('_')
            if len(split_col) != 1:
                if side == 'right':
                    if split_col[1] in ['eye','ear','shoulder','elbow','wrist','arm']:
                        if split_col[0] == 'right':
                            split_col[0] = 'lead'
                        elif split_col[0] == 'left':
                            split_col[0] = 'rear'
                
                    elif split_col[1] in ['hip','knee','ankle','heel']:
                        if split_col[0] == 'right':
                            split_col[0] = 'rear'
                        elif split_col[0] == 'left':
                            split_col[0] = 'lead'
                            
                if side == 'left':
                    if split_col[1] in ['eye','ear','shoulder','elbow','wrist','arm']:
                        if split_col[0] == 'right':
                            split_col[0] = 'rear'
                        elif split_col[0] == 'left':
                            split_col[0] = 'lead'
                
                    elif split_col[1] in ['hip','knee','ankle','heel']:
                        if split_col[0] == 'right':
                            split_col[0] = 'lead'
                        elif split_col[0] == 'left':
                            split_col[0] = 'rear'
                            
            else:
                split_col = split_col
            
            split_col = '_'.join(split_col)
            newcols.append(split_col)
            
        d.columns = newcols
        
        
        change_axis_col = []
        if side == 'left':
            for col in d.columns:
                if '3d' in col:
                    split_col = col.split('_')
                    if split_col[-1] == 'x':
                        split_col[-1] = 'z'
                    elif split_col[-1] == 'z':
                        split_col[-1] = 'x'
                    split_col = '_'.join(split_col)
                    change_axis_col.append(split_col)
                    
                else:
                    change_axis_col.append(col)
                    
            d.columns = change_axis_col
            
            
        x = (d['lead_wrist_jc_3d_x'] - d['lead_ear_3d_x']) ** 2
        z = (d['lead_wrist_jc_3d_z'] - d['lead_ear_3d_z']) ** 2
        
        xz = np.sqrt(x + z)
        d['head_hand_distance'] = xz
        
        hor_plane = np.sqrt((d['rear_knee_jc_3d_x'] - d['rear_ankle_jc_3d_x'])**2 + (d['rear_knee_jc_3d_z'] - d['rear_ankle_jc_3d_z'])**2)
        ver_axis = d['rear_knee_jc_3d_y'] - d['rear_ankle_jc_3d_y']
        shank_angle = np.arctan2(ver_axis, hor_plane)
        shank_angle = np.rad2deg(shank_angle)
        d['shank_angle'] = shank_angle
        
        usecols = [
        'lead_knee_jc_3d_y','pelvis_3d_y',
        'pelvis_rotational_velocity_with_respect_to_ground',
        'trunk_rotational_velocity_with_respect_to_ground',
        'lead_arm_rotational_velocity_with_respect_to_ground',
        'lead_elbow_flexion_velocity',
        'lead_shoulder_external_rotation_velocity',
        'lead_knee_extension','lead_knee_extension_velocity',
        'lead_shoulder_adduction','lead_shoulder_external_rotation','lead_elbow_flexion',
        'trunk_twist_clockwise',  'trunk_lateral_flexion_right', 'lead_shoulder_horizontal_adduction','head_hand_distance','shank_angle'
        ]
        
        d = d[usecols]
        
        d['lead_shoulder_adduction'] = -d['lead_shoulder_adduction']
        d['lead_knee_extension'] = -d['lead_knee_extension']
        d['lead_elbow_flexion_velocity'] = - d['lead_elbow_flexion_velocity']
        if side == 'left':
            d['pelvis_rotational_velocity_with_respect_to_ground'] = - d['pelvis_rotational_velocity_with_respect_to_ground']
            d['trunk_rotational_velocity_with_respect_to_ground'] = - d['trunk_rotational_velocity_with_respect_to_ground']
            d['lead_arm_rotational_velocity_with_respect_to_ground'] = - d['lead_arm_rotational_velocity_with_respect_to_ground']
            d['trunk_twist_clockwise'] = - d['trunk_twist_clockwise']
        
        if side == 'right':
            d['trunk_lateral_flexion_right'] = - d['trunk_lateral_flexion_right']
        plot = False
        try:
            kh_time = np.where(d['lead_knee_jc_3d_y'].iloc[:fc_frame] == d['lead_knee_jc_3d_y'].iloc[:fc_frame].max())[0][0]
            mer_time = detect_peaks(d['lead_shoulder_external_rotation'].iloc[:br_frame], mph = 130, mpd = 480, show=plot)[0]
            d['time'] = np.arange(0, len(d)/240, 1/240)[:len(d)]
            d['name'] = name
            d['side'] = side
            d['trial'] = trial
            d['weight'] = weight
            d['height'] = height
            d['kh_time'] = kh_time
            d['fc_time'] = fc_frame
            d['mer_time'] = mer_time
            d['br_time'] = br_frame
            d['ball_vel'] = ball_vel
            
            d = d.iloc[:-1,:]
            data[f'{name}_{trial}'] = d
            if kh_time < fc_frame < mer_time < br_frame:
                d.to_csv(f"data/{day}/process/{name}_{trial}_{side}_{weight}_{height}_{ball_vel}.csv",index=False)
                print(f'{name}_{trial} 저장 완료')
            else:
                print('============================')
                print(f'{name}_{trial} 시점 오류')
                print('============================')
                wrong.append(f'{name}_{trial}')
        except Exception as e:
            
            print('============================')
            print(e)
            print(f'{name}_{trial} 시점 오류')
            print('============================')
            wrong.append(f'{name}_{trial}')

n51_011 저장 완료
n51_005 저장 완료
n51_008 저장 완료
n52_004 저장 완료
n52_005 저장 완료
n52_006 저장 완료
n53_011 저장 완료
n53_024 저장 완료
n53_009 저장 완료
n54_027 시점 오류
n54_029 저장 완료
n54_008 시점 오류
n65_014 저장 완료
n65_004 저장 완료
n65_006 저장 완료
n55_001 저장 완료
n55_005 저장 완료
n55_006 저장 완료
n64_010 저장 완료
n64_014 저장 완료
n64_008 저장 완료
n57_001 저장 완료
n57_003 저장 완료
n57_009 저장 완료
n63_016 저장 완료
n63_017 저장 완료
n63_019 저장 완료
n58_001 저장 완료
n58_015 저장 완료
n58_006 저장 완료
n62_013 저장 완료
n62_003 저장 완료
n62_005 저장 완료
n61_001 저장 완료
n61_020 저장 완료
n61_003 저장 완료
n60_012 저장 완료
n60_017 저장 완료
n60_006 저장 완료
n59_014 저장 완료
n59_015 저장 완료
n59_002 저장 완료
n56_013 저장 완료
n56_004 저장 완료
n56_005 저장 완료
n66_013 저장 완료
n66_016 저장 완료
n66_028 저장 완료


In [6]:
wrong

['n54_027', 'n54_008']

In [5]:
wrong = []
data = {}
for file in file_list:
    df = pd.read_csv(file)
    sessions = df.sessionid.unique()

    for session in sessions:
        d = df[df['sessionid'] == session].sort_values('frame').reset_index(drop=True)
        side = d['handedness'][0]
        trial = str(session.split('_')[-1]).zfill(3)
        name = d['athlete_name'][0].replace(' ','')
        
        weight, height, fc_frame, br_frame, ball_vel, _ = players.loc[f"{name}_{trial}"]
        fc_frame = int(fc_frame) ; br_frame = int(br_frame)
        weight = round(weight)
        height = round(height)
        newcols = []
        for col in d.columns:
            split_col = col.split('_')
            if len(split_col) != 1:
                if side == 'right':
                    if split_col[1] in ['eye','ear','shoulder','elbow','wrist','arm']:
                        if split_col[0] == 'right':
                            split_col[0] = 'lead'
                        elif split_col[0] == 'left':
                            split_col[0] = 'rear'
                
                    elif split_col[1] in ['hip','knee','ankle','heel']:
                        if split_col[0] == 'right':
                            split_col[0] = 'rear'
                        elif split_col[0] == 'left':
                            split_col[0] = 'lead'
                            
                if side == 'left':
                    if split_col[1] in ['eye','ear','shoulder','elbow','wrist','arm']:
                        if split_col[0] == 'right':
                            split_col[0] = 'rear'
                        elif split_col[0] == 'left':
                            split_col[0] = 'lead'
                
                    elif split_col[1] in ['hip','knee','ankle','heel']:
                        if split_col[0] == 'right':
                            split_col[0] = 'lead'
                        elif split_col[0] == 'left':
                            split_col[0] = 'rear'
                            
            else:
                split_col = split_col
            
            split_col = '_'.join(split_col)
            newcols.append(split_col)
            
        d.columns = newcols
        
        
        change_axis_col = []
        if side == 'left':
            for col in d.columns:
                if '3d' in col:
                    split_col = col.split('_')
                    if split_col[-1] == 'x':
                        split_col[-1] = 'z'
                    elif split_col[-1] == 'z':
                        split_col[-1] = 'x'
                    split_col = '_'.join(split_col)
                    change_axis_col.append(split_col)
                    
                else:
                    change_axis_col.append(col)
                    
            d.columns = change_axis_col
            
            
        x = (d['lead_wrist_jc_3d_x'] - d['lead_ear_3d_x']) ** 2
        z = (d['lead_wrist_jc_3d_z'] - d['lead_ear_3d_z']) ** 2
        
        xz = np.sqrt(x + z)
        d['head_hand_distance'] = xz
        
        hor_plane = np.sqrt((d['rear_knee_jc_3d_x'] - d['rear_ankle_jc_3d_x'])**2 + (d['rear_knee_jc_3d_z'] - d['rear_ankle_jc_3d_z'])**2)
        ver_axis = d['rear_knee_jc_3d_y'] - d['rear_ankle_jc_3d_y']
        shank_angle = np.arctan2(ver_axis, hor_plane)
        shank_angle = np.rad2deg(shank_angle)
        d['shank_angle'] = shank_angle
        
        usecols = [
        'lead_knee_jc_3d_y','pelvis_3d_y',
        'pelvis_rotational_velocity_with_respect_to_ground',
        'trunk_rotational_velocity_with_respect_to_ground',
        'lead_arm_rotational_velocity_with_respect_to_ground',
        'lead_elbow_flexion_velocity',
        'lead_shoulder_external_rotation_velocity',
        'lead_knee_extension','lead_knee_extension_velocity',
        'lead_shoulder_adduction','lead_shoulder_external_rotation','lead_elbow_flexion',
        'trunk_twist_clockwise',  'trunk_lateral_flexion_right', 'lead_shoulder_horizontal_adduction','head_hand_distance','shank_angle'
        ]
        
        d = d[usecols]
        
        d['lead_shoulder_adduction'] = -d['lead_shoulder_adduction']
        d['lead_knee_extension'] = -d['lead_knee_extension']
        d['lead_elbow_flexion_velocity'] = - d['lead_elbow_flexion_velocity']
        if side == 'left':
            d['pelvis_rotational_velocity_with_respect_to_ground'] = - d['pelvis_rotational_velocity_with_respect_to_ground']
            d['trunk_rotational_velocity_with_respect_to_ground'] = - d['trunk_rotational_velocity_with_respect_to_ground']
            d['lead_arm_rotational_velocity_with_respect_to_ground'] = - d['lead_arm_rotational_velocity_with_respect_to_ground']
            d['trunk_twist_clockwise'] = - d['trunk_twist_clockwise']
        
        if side == 'right':
            d['trunk_lateral_flexion_right'] = - d['trunk_lateral_flexion_right']

        plot = False
        kh_time = np.where(d['lead_knee_jc_3d_y'].iloc[:fc_frame] == d['lead_knee_jc_3d_y'].iloc[:fc_frame].max())[0][0]
        mer_time = detect_peaks(d['lead_shoulder_external_rotation'].iloc[:br_frame], mph = 130, mpd = 480, show=plot)[0]
        
        
        d['time'] = np.arange(0, len(d)/240, 1/240)[:len(d)]
        d['name'] = name
        d['side'] = side
        d['trial'] = trial
        d['weight'] = weight
        d['height'] = height
        d['kh_time'] = kh_time
        d['fc_time'] = fc_frame
        d['mer_time'] = mer_time
        d['br_time'] = br_frame
        d['ball_vel'] = ball_vel
        
        if (name == 'n54') & (trial == '027'):
            mer_time = 602
            d['mer_time'] = mer_time
        elif (name == 'n54') & (trial == '029'):
            mer_time = 449
            d['mer_time'] = mer_time
        elif (name == 'n54') & (trial == '008'):
            mer_time = 624
            d['mer_time'] = mer_time
        
        d = d.iloc[:-1,:]
        data[f'{name}_{trial}'] = d
        if kh_time < fc_frame < mer_time < br_frame:
            d.to_csv(f"data/{day}/process/{name}_{trial}_{side}_{weight}_{height}_{ball_vel}.csv",index=False)
            print(f'{name}_{trial} 저장 완료')
        else:
            print('============================')
            print(f'{name}_{trial} 시점 오류')
            print('============================')
            wrong.append(f'{name}_{trial}')


n51_011 저장 완료
n51_005 저장 완료
n51_008 저장 완료
n52_004 저장 완료
n52_005 저장 완료
n52_006 저장 완료
n53_011 저장 완료
n53_024 저장 완료
n53_009 저장 완료
n54_027 저장 완료
n54_029 저장 완료
n54_008 저장 완료
n65_014 저장 완료
n65_004 저장 완료
n65_006 저장 완료
n55_001 저장 완료
n55_005 저장 완료
n55_006 저장 완료
n64_010 저장 완료
n64_014 저장 완료
n64_008 저장 완료
n57_001 저장 완료
n57_003 저장 완료
n57_009 저장 완료
n63_016 저장 완료
n63_017 저장 완료
n63_019 저장 완료
n58_001 저장 완료
n58_015 저장 완료
n58_006 저장 완료
n62_013 저장 완료
n62_003 저장 완료
n62_005 저장 완료
n61_001 저장 완료
n61_020 저장 완료
n61_003 저장 완료
n60_012 저장 완료
n60_017 저장 완료
n60_006 저장 완료
n59_014 저장 완료
n59_015 저장 완료
n59_002 저장 완료
n56_013 저장 완료
n56_004 저장 완료
n56_005 저장 완료
n66_013 저장 완료
n66_016 저장 완료
n66_028 저장 완료


In [14]:
for session in data:
    if session in wrong:
        d = data[session]
        weight, height, fc_frame, br_frame, ball_vel = players.loc[f"{name}_{trial}"]
        fc_frame = int(fc_frame) ; br_frame = int(br_frame)
        kh_time = detect_peaks(d['lead_knee_jc_3d_y'].iloc[:fc_frame], mph = 0.1, mpd = 480, show=plot)[0]
        mer_time = detect_peaks(d['lead_shoulder_external_rotation'].iloc[:br_frame], mph = 130, mpd = 480, show=plot)[0]
        d['time'] = np.arange(0, len(d)/240, 1/240)
        d['name'] = name
        d['side'] = side
        d['trial'] = trial
        d['weight'] = weight
        d['height'] = height
        d['kh_time'] = kh_time
        d['fc_time'] = fc_frame
        d['mer_time'] = mer_time
        d['br_time'] = br_frame
        d['ball_vel'] = ball_vel